In [2]:
import numpy as np
import tensorflow as tf
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.image import load_img

from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.applications.xception import decode_predictions

from keras_preprocessing.image import ImageDataGenerator


SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [8]:
from PIL import Image
import os

directories = ['./data/train', './data/test']

for directory in directories:
    for file_name in os.listdir(directory):
        file_path = os.path.join(directory, file_name)
        
        # Skip if it's a directory
        if os.path.isdir(file_path):
            continue
        
        try:
            with Image.open(file_path) as img:
                img.verify()  # This checks if the file is a valid image.
        except (IOError, SyntaxError) as e:
            print(f"Removing corrupted image: {file_path}")
            os.remove(file_path)  # Remove or log problematic files

In [10]:
input_size = 200
inputs = keras.Input(shape=(input_size, input_size, 3))

conv2D = keras.layers.Conv2D(
    filters=32,
    kernel_size=(3, 3),
    activation='relu'
)(inputs)

max_pool = keras.layers.MaxPooling2D(
    pool_size=(2, 2)
)(conv2D)

vectors = keras.layers.Flatten()(max_pool)

size_inner = 64 
inner = keras.layers.Dense(size_inner, activation='relu')(vectors)

size_out = 1
outputs = keras.layers.Dense(size_out, activation='sigmoid')(inner)

# Build the model
lr = 0.002
momentum = 0.8
model = keras.Model(inputs, outputs)

optimizer = keras.optimizers.SGD(
    learning_rate=lr,
    momentum=momentum
)
loss = keras.losses.BinaryCrossentropy()

model.compile(
    optimizer=optimizer, 
    loss=loss, 
    metrics=['accuracy']
)

train_ds = tf.keras.utils.image_dataset_from_directory(
    './data/train',
    labels='inferred',
    label_mode='binary',
    batch_size=20,
    image_size=(input_size, input_size),
    shuffle=True
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    './data/test',
    labels='inferred',
    label_mode='binary',
    batch_size=20,
    image_size=(input_size, input_size),
    shuffle=True

)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

Found 801 files belonging to 2 classes.
Found 201 files belonging to 2 classes.
Epoch 1/10
 6/41 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.4349 - loss: 190865.0938

2024-12-06 21:28:57.701259: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]


InvalidArgumentError: Graph execution error:

Detected at node decode_image/DecodeImage defined at (most recent call last):
<stack traces unavailable>
Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_multi_step_on_iterator_2252]